In [ ]:
!pip install langchain_community langchain_openai

In [ ]:
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY_OPENAI")

In [ ]:
# 객체 생성
llm = ChatOpenAI(
    model_name = "gpt-4o",
    temperature = 0.1,
    max_tokens = 2048
)

In [ ]:
# query 내용
question = "세종대왕이 누구인지 설명해주세요"

# query
result = llm.invoke(question)
print(result.content)

In [ ]:
# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명해주세요"

# 템플릿 완성
prompt = PromptTemplate(
    template = template,
    input_variables = ['who']
)

print(prompt)

In [ ]:
# 연결된 chain 생성
llm_chain = prompt | llm

result = llm_chain.invoke({"who" : "이순신 장군"})
print(result.content)

In [ ]:
result = llm_chain.invoke(
    {"who" : "이순신 장군"},
    config = {"collbacks" : [ConsoleCallbackHandler()]}
)

In [ ]:
print(result)

# RunnableWithMessageHistory 과거 대화 내역 반영

In [ ]:
# 프롬프트 템플릿 생성
template = '''아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 코랩봄입니다.
대화 문맥을 바탕으로 친절한 답변을 진행하세요.

Current Conversation : {history}

Human : {input}

AI : '''

prompt = PromptTemplate(
    template = template,
    input_variables = ['history', 'input']
)

# 객체를 새로 선언하고 prompt와 llm 객체를 연결합니다.
llm = ChatOpenAI(model_name = 'gpt-4o')

llm_chain = prompt | llm

In [ ]:
store = {}
session_id = 'test'

In [ ]:
if session_id not in store:
  store[session_id] = ChatMessageHistory()

session_history = store[session_id]

In [ ]:
message_history = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id : session_history,
    input_messages_key = "input",
    history_messages_key = "history"
)

In [ ]:
result = message_history.invoke(
    {"input" : "당신은 어디에서 만들었습니까?"},
    config = {"configurable" : {"session_id" : "test"}}
)

print(result.content)

In [ ]:
result = message_history.invoke(
    {"input" : "푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요"},
    config = {"configurable" : {"session_id" : "test"}}
)

print(result.content)

In [ ]:
result = message_history.invoke(
    {"input" : "석양을 주제로도 해줘"},
    config = {"configurable" : {"session_id" : "test"}}
)

print(result.content)

In [ ]:
print(store)